In [18]:
# import packages
import pandas as pd
import numpy as np
import biogeme as biogeme
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import biogeme.messaging as msgnotes
from biogeme.expressions import Beta
from biogeme.expressions import (
    Beta,
    DefineVariable,
    bioDraws,
    PanelLikelihoodTrajectory,
    MonteCarlo,
    log,
    Derive,
    bioNormalCdf,
    Elem
)
import math
from datetime import datetime
import matplotlib.pyplot as plt
from factor_analyzer import FactorAnalyzer

In [19]:
# import psychometric variables data
psych_vars = pd.read_csv('Data/psych_vars.csv')

# drop NA responses [3]
psych_vars = psych_vars.dropna()
psych_vars.isnull().sum().sum()

0

In [20]:
# rename columns from question numbers to variables

psych_vars.rename(columns={'q16':'socialnorm1', 'q17':'socialnorm2', 'q18':'socialnorm3', 'q19':'socialnorm4', 'q20':'socialnorm5', 'q21':'socialnorm6'}, inplace=True)
psych_vars.rename(columns={'q22': 'expectedperf1', 'q23': 'expectedperf2', 'q24': 'expectedperf3', 'q25': 'expectedperf4', 'q26': 'expectedperf5'}, inplace=True)
psych_vars.rename(columns={'q27': 'envbenefit1', 'q28': 'envbenefit2', 'q29': 'envbenefit3', 'q30': 'envbenefit4'}, inplace=True)
psych_vars.rename(columns={'q31': 'perceivedrisk1', 'q32': 'perceivedrisk2', 'q33': 'perceivedrisk3'}, inplace=True)
psych_vars.rename(columns={'q34': 'resideffects1', 'q35': 'resideffects2', 'q36': 'resideffects3', 'q37': 'resideffects4'}, inplace=True)
psych_vars.rename(columns={'q38': 'envvalues1', 'q39': 'envvalues2', 'q40': 'envvalues3', 'q41': 'envvalues4'}, inplace=True)
psych_vars.rename(columns={'q42': 'antimicro1', 'q43': 'antimicro2', 'q44': 'antimicro3'}, inplace=True)
psych_vars.rename(columns={'q45': 'intentiontouse1', 'q46': 'intentiontouse2', 'q47': 'intentiontouse3'}, inplace=True)

# convert responses into numerical values with 6 as "positive" and "1" as negative attitudes towards micromobility

# socialnorm1 --> "my family or friends think using bikesharing or scootersharing is a positive thing"
psych_vars.replace({'socialnorm1': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# socialnorm 2 --> "people important to me think that using bikesharing or scootersharing is a positive thing"
psych_vars.replace({'socialnorm2': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# socialnorm3 --> "in the near future more people in my city will use bikesharing or scootersharing"
psych_vars.replace({'socialnorm3': {'Considerably fewer': 1, 'Fewer': 2, 'Slightly fewer': 3, 'Slightly more': 4, 'More': 5, 'A lot more': 6}}, inplace=True)
# socialnorm4 --> "people who are important to me think that I should use bikesharing or scooter sharing"
psych_vars.replace({"socialnorm4": {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# socialnorm5 --> "it is a shame to use bikeshaing or scootersharing"
psych_vars.replace({'socialnorm5': {'Strongly agree': 1, 'Agree': 2, 'Slightly agree': 3, 'Slightly disagree': 4, 'Disagree': 5, 'Strongly disagree': 6}}, inplace=True)
# socialnorm6 --> "the social media evaluates bikesharing or scootersharing negatively"
psych_vars.replace({'socialnorm6': {'Very negative': 1, 'Negative': 2, 'Somewhat negative': 3, 'Somewhat positive': 4, 'Positive': 5, 'Very positive': 6}}, inplace=True)
# expectedperf1 --> "shared micromobility is convenient"
psych_vars.replace({'expectedperf1': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# expected perf2 --> "shared micromobility is effective for my personal mobility"
psych_vars.replace({'expectedperf2': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# expectedperf3 --> "shared micromobility can help me reach my destination efficiently"
psych_vars.replace({'expectedperf3': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# expectedperf4 --> "there are enough shared bikes/scooters available whenever I want to use them"
psych_vars.replace({'expectedperf4': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# expectedperf5 --> "I can comfortably take rides on a shared bike or scooter for my daily business"
psych_vars.replace({'expectedperf5': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# envbenefit1 --> "using shared micromobility will help alleviate traffic congestion"
psych_vars.replace({'envbenefit1': {'Very unlikely': 1, 'Unlikely': 2, 'Slightly unlikely': 3, 'Slightly likely': 4, 'Likely': 5, 'Very likely': 6}}, inplace=True)
# envbenefit2 --> "using shared micromobility will reduce carbon emission and air pollution"
psych_vars.replace({'envbenefit2': {'Very unlikely': 1, 'Unlikely': 2, 'Slightly unlikely': 3, 'Slightly likely': 4, 'Likely': 5, 'Very likely': 6}}, inplace=True)
# envbenefit3 --> "using a shared bike or scooter fits my environmental concerns"
psych_vars.replace({'envbenefit3': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# envbenefit4 --> "shared micromobility has a positive impact on urban traffic"
psych_vars.replace({'envbenefit4': {'Significantly increase': 1, 'Increase': 2, 'Slightly increase': 3, 'Slightly decrease': 4, 'Decrease': 5, 'Significantly decrease': 6}}, inplace=True)
# perceivedrisk1 --> "I would feel safe riding a shared bike or scooter in traffic"
psych_vars.replace({'perceivedrisk1': {'Very unsafe': 1, 'Unsafe': 2, 'Somewhat unsafe': 3, 'Somewhat safe': 4, 'Safe': 5, 'Very safe': 6}}, inplace=True)
# perceivedrisk2 --> "I think riding a shared bike or scooter is dangerous"
psych_vars.replace({'perceivedrisk2': {'Very dangerous': 1, 'Dangerous': 2, 'Somewhat dangerous': 3, 'Somewhat safe': 4, 'Safe': 5, 'Very safe': 6}}, inplace=True)
# perceivedrisk3 --> "I would feel nervous about having an accident when riding a shared bike or scooter"
psych_vars.replace({'perceivedrisk3': {'Strongly agree': 1, 'Agree': 2, 'Slightly agree': 3, 'Slightly disagree': 4, 'Disagree': 5, 'Strongly disagree': 6}}, inplace=True)
# resideffects1 --> "I knew about bikesharing or scootersharing before"
psych_vars.replace({'resideffects1': {'No': 0, 'Yes': 1}}, inplace=True)
# resideffects2 --> "many people around me know about bikesharing or scootersharing"
psych_vars.replace({'resideffects2': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# resideffects3 --> "I have used bikesharing or scootersharing before"
psych_vars.replace({'resideffects3': {'No': 0, 'Yes': 1}}, inplace=True)
# resideffects4 --> "there are bikesharing or scootersharing available to me and I can use them regularly"
psych_vars.replace({'resideffects4': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# envvalues1 --> "I would like to do my part to reduce carbon emission and air pollution"
psych_vars.replace({'envvalues1': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# envvalues2 --> "I always consider how my transport choices affect the environment"
psych_vars.replace({'envvalues2': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# envvalues3 --> "I consider myself to be an environmentally conscious person"
psych_vars.replace({'envvalues3': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# envvalues4 --> "global warming is fake science"
psych_vars.replace({'envvalues4': {'Strongly agree': 1, 'Agree': 2, 'Slightly agree': 3, 'Slightly disagree': 4, 'Disagree': 5, 'Strongly disagree': 6}}, inplace=True)
# F11 --> "shared micromobility is a very bad idea"
psych_vars.replace({'antimicro1': {'Strongly agree': 1, 'Agree': 2, 'Slightly agree': 3, 'Slightly disagree': 4, 'Disagree': 5, 'Strongly disagree': 6}}, inplace=True)
# F12 --> "shared micromobility causes a lot of problems"
psych_vars.replace({'antimicro2': {'Strongly agree': 1, 'Agree': 2, 'Slightly agree': 3, 'Slightly disagree': 4, 'Disagree': 5, 'Strongly disagree': 6}}, inplace=True)
# F13 --> "shared micromobility should not have existed in cities"
psych_vars.replace({'antimicro3': {'Strongly agree': 1, 'Agree': 2, 'Slightly agree': 3, 'Slightly disagree': 4, 'Disagree': 5, 'Strongly disagree': 6}}, inplace=True)
# intentiontouse1 --> "I'm willing to use bikesharing or scootersharing in the future"
psych_vars.replace({'intentiontouse1': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# intentiontouse2 --> "I would recommend friends and family to use bikesharing or scootersharing"
psych_vars.replace({'intentiontouse2': {'Definetely would not': 1, 'Probably would not': 2, 'Maybe would not': 3, 'Maybe would': 4, 'Probably would': 5, 'Definetely would': 6}}, inplace=True)
# intentiontouse3 --> "I'm willing to use bikesharing or scootersharing on a regular basis"
psych_vars.replace({'intentiontouse3': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)


In [21]:
# create new dataframes for factor analysis with and without user column since FA package does not work with user column
factor_df_with_user = psych_vars[['user', 'socialnorm1', 'socialnorm2', 'socialnorm3', 'socialnorm4', 'socialnorm5', 'socialnorm6', 'expectedperf1', 'expectedperf2', 'expectedperf3', 'expectedperf4', 'expectedperf5', 'envbenefit1', 'envbenefit2', 'envbenefit3', 'envbenefit4', 'perceivedrisk1', 'perceivedrisk2', 'perceivedrisk3', 'resideffects1', 'resideffects2', 'resideffects3', 'resideffects4', 'envvalues1', 'envvalues2', 'envvalues3', 'envvalues4', 'antimicro1', 'antimicro2', 'antimicro3', 'intentiontouse1', 'intentiontouse2', 'intentiontouse3']]
factor_df = psych_vars[['socialnorm1', 'socialnorm2', 'socialnorm3', 'socialnorm4', 'socialnorm5', 'socialnorm6', 'expectedperf1', 'expectedperf2', 'expectedperf3', 'expectedperf4', 'expectedperf5', 'envbenefit1', 'envbenefit2', 'envbenefit3', 'envbenefit4', 'perceivedrisk1', 'perceivedrisk2', 'perceivedrisk3', 'resideffects1', 'resideffects2', 'resideffects3', 'resideffects4', 'envvalues1', 'envvalues2', 'envvalues3', 'envvalues4', 'antimicro1', 'antimicro2', 'antimicro3', 'intentiontouse1', 'intentiontouse2', 'intentiontouse3']]

# check
print(factor_df.head())

# perform factor analysis with promax rotation, 6 factors, maximum likelihood method
fa = FactorAnalyzer(rotation='promax', n_factors=6, method = 'ml')
fa.fit(factor_df)

# store the factor scores
scores = fa.transform(factor_df)

# extract the user column
user_column = factor_df_with_user['user']

# create a new DataFrame with factor scores and 'user' column
factor_scores_df = pd.DataFrame(scores, columns=['Factor_1', 'Factor_2', 'Factor_3', 'Factor_4', 'Factor_5', 'Factor_6']) #, 'Factor_7', 'Factor_8'])
factor_scores_df['user'] = user_column

   socialnorm1  socialnorm2  socialnorm3  socialnorm4  socialnorm5  \
0            4            4            5            4            6   
1            5            5            4            5            6   
2            5            5            6            2            6   
3            4            4            5            3            6   
4            4            4            4            2            6   

   socialnorm6  expectedperf1  expectedperf2  expectedperf3  expectedperf4  \
0            4              5              4              4              1   
1            4              5              5              5              3   
2            6              6              2              2              4   
3            5              5              4              4              2   
4            4              5              1              2              2   

   ...  envvalues1  envvalues2  envvalues3  envvalues4  antimicro1  \
0  ...           5           5          

In [22]:
# merge factor_scores_df and psych_vars on 'user'
psych_vars_with_factors = pd.merge(psych_vars, factor_scores_df, on = 'user')

In [23]:
# import micromobility stated preference experiment data
micro_pool_socio = pd.read_csv('Data/micro_pool_socio_bio2up.csv')

In [24]:
# Import scoot_user_id.csv
scoot_user_id = pd.read_csv('Data/scoot_user_id.csv')

# merge micro_pool_socio and scooter_user_id on 'p'
micro_pool_socio = pd.merge(micro_pool_socio, scoot_user_id, on='p')


In [25]:
# filter dataset for only micromobility mode choices

micro0to5 = (((
# 0-2 miles first choice
micro_pool_socio['scooter_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 1) & (
    micro_pool_socio['dkbike_av'] == 1) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 0)) | ((
# 0-2 miles second choice
    # scooter_av == 1 & dlbike_av == 1 & everything else == 0
    micro_pool_socio['scooter_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 1) & (
    micro_pool_socio['dkbike_av'] == 0) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 0)) | ((
    #'scooter'_av ==1 & 'dk_bike'_av == 1 & everything else == 0
    micro_pool_socio['scooter_av'] == 1) & (
    micro_pool_socio['dkbike_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 0) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 0)) | ((
    #'dk_bike_av' == 1 & 'dl_bike_av' == 1 & everything else == 0
    micro_pool_socio['dkbike_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 1) & (
    micro_pool_socio['scooter_av'] == 0) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 0
    )) | ((
# 2-5 miles
    micro_pool_socio['scooter_av'] == 1) & (
    micro_pool_socio['dlbike_av'] == 0) & (
    micro_pool_socio['dkbike_av'] == 0) & (
    micro_pool_socio['car_av'] == 0) & (
    micro_pool_socio['transit_av'] == 0) & (
    micro_pool_socio['rd_av'] == 0) & (
    micro_pool_socio['walk_av'] == 0) & (
    micro_pool_socio['bike_av'] == 0) & (
    micro_pool_socio['sctransit_av'] == 1
)))

micro_0_to_5 = micro_pool_socio.loc[micro0to5]

In [26]:
# merge psych_vars and micro_pool_socio using inner join, thus dropping any users who did not answer psychometric questions
combined_psych = pd.merge(micro_0_to_5, psych_vars_with_factors, on='user', )

# get count of unique users
combined_psych['user'].nunique()

# sort by dperson
combined_psych = combined_psych.sort_values(by=['who'])

# drop user column bc is it is type string
combined_psych = combined_psych.drop(columns=['user'])

# convert to database
database = db.Database('combined_psych', combined_psych)
# panel data
database.panel("who")
# create global variables
globals().update(database.variables)

In [27]:
# print the number of unique entries in the "who" column of combined_psych
print(combined_psych['who'].nunique())

853


In [28]:
# create a new dataframe with only the unique 'who' values in combined_psych
respondents = combined_psych.drop_duplicates(subset=['who']).copy()

total_rows = (combined_psych['who'].nunique())

# calculate summary statistics for 'age' variable
age_min = respondents['age'].min()
age_median = respondents['age'].median()
age_mean = respondents['age'].mean()
age_max = respondents['age'].max()

# print the summary statistics
print(f"Min: {age_min}")
print(f"Median: {age_median}")
print(f"Mean: {age_mean:.2f}")
print(f"Max: {age_max}")

# calculate summary statistics for the remaining socio-demographic variables
gender_F_percentage = (respondents['gender_F'].sum() / total_rows * 100)
gender_M_percentage = (respondents['gender_M'].sum() / total_rows) * 100
gender_O_percentage = (respondents['gender_O'].sum() / total_rows) * 100
race_white_percentage = (respondents['race'] == 1).sum() / total_rows * 100
race_black_percentage = (respondents['race'] == 2).sum() / total_rows * 100
race_american_indian_percentage = (respondents['race'] == 3).sum() / total_rows * 100
race_asian_percentage = (respondents['race'] == 4).sum() / total_rows * 100
race_pacific_percentage = (respondents['race'] == 5).sum() / total_rows * 100
education_less_than_bach = (respondents['edu'] < 5).sum() / total_rows * 100
education_bachelor_higher_percentage = (respondents['edu'] >= 5).sum() / total_rows * 100
employment_employed_percentage = (respondents['work'] <= 2).sum() / total_rows * 100
employment_not_employed_percentage = (respondents['work'] > 2).sum() / total_rows * 100
income_under_25k_percentage = (respondents['hhincome'] <= 4).sum() / total_rows * 100
income_25k_50k_percentage = ((respondents['hhincome'] == 5) | (respondents['hhincome'] == 6)).sum() / total_rows * 100
income_50k_75k_percentage = ((respondents['hhincome'] == 7)).sum() / total_rows * 100
income_75k_100k_percentage = (respondents['hhincome'] == 8).sum() / total_rows * 100
income_100k_150k_percentage = (respondents['hhincome'] == 9).sum() / total_rows * 100
income_above_150k_percentage = (respondents['hhincome'] >= 10).sum() / total_rows * 100


# print the summary statistics
print(f"Female: {gender_F_percentage:.1f}%")
print(f"Male: {gender_M_percentage:.1f}%")
print(f"Another: {gender_O_percentage:.1f}%")
print(f"White: {race_white_percentage:.1f}%")
print(f"Black or African American: {race_black_percentage:.1f}%")
print(f"American Indian or Alaska Native: {race_american_indian_percentage:.1f}%")
print(f"Asian: {race_asian_percentage:.1f}%")
print(f"Native Hawaiian or Other Pacific Islander: {race_pacific_percentage:.1f}%")
print(f"Less than bachelor’s degree: {education_less_than_bach:.1f}%")
print(f"Bachelor’s degree or higher: {education_bachelor_higher_percentage:.1f}%")
print(f"Employed: {employment_employed_percentage:.1f}%")
print(f"Not employed: {employment_not_employed_percentage:.1f}%")
print(f"Under $25,000: {income_under_25k_percentage:.1f}%")
print(f"$25,000-$49,999: {income_25k_50k_percentage:.1f}%")
print(f"$50,000-$74,999: {income_50k_75k_percentage:.1f}%")
print(f"$75,000-$99,999: {income_75k_100k_percentage:.1f}%")
print(f"$100,000-$149,999: {income_100k_150k_percentage:.1f}%")
print(f"≥  $150,000: {income_above_150k_percentage:.1f}%")

Min: 18
Median: 35.0
Mean: 37.97
Max: 84
Female: 42.7%
Male: 56.6%
Another: 0.7%
White: 80.8%
Black or African American: 8.8%
American Indian or Alaska Native: 1.3%
Asian: 6.4%
Native Hawaiian or Other Pacific Islander: 0.1%
Less than bachelor’s degree: 31.7%
Bachelor’s degree or higher: 68.3%
Employed: 89.6%
Not employed: 10.4%
Under $25,000: 15.8%
$25,000-$49,999: 25.9%
$50,000-$74,999: 25.9%
$75,000-$99,999: 16.6%
$100,000-$149,999: 9.8%
≥  $150,000: 5.9%


In [29]:

B_SCOOTERTIME = Beta('B_SCOOTERTIME', 0, None, None, 0)
B_BIKETIME = Beta('B_BIKETIME', 0, None, None, 0)
B_SCTRANSITTIME = Beta('B_SCTRANSITTIME', 0, None, None, 0)

B_ACCESS = Beta('B_ACCESS', 0, None, None, 0)
B_DROP =Beta('B_DROP', 0, None, None, 0)
B_WAITAV = Beta('B_WAITAV', 0, None, None, 0)
B_AV = Beta('B_AV', 0, None, None, 0)

B_COST_ADJ = Beta('B_COST_ADJ', 0, None, None, 0)

In [30]:
# intercepts
ASC_SCOOTER = Beta('ASC_SCOOTER', 0, None, None, 1)

ASC_DKBIKE = Beta('ASC_DKBIKE', 0, None, None, 0)
ASC_DKBIKE_S = Beta('ASC_DKBIKE_S', 1, None, None, 0)
ASC_DKBIKE_RND = ASC_DKBIKE + ASC_DKBIKE_S * bioDraws('ASC_DKBIKE_RND', 'NORMAL_ANTI')

ASC_DLBIKE = Beta('ASC_DLBIKE', 0, None, None, 0)
ASC_DLBIKE_S = Beta('ASC_DLBIKE_S', 1, None, None, 0)
ASC_DLBIKE_RND = ASC_DLBIKE + ASC_DLBIKE_S * bioDraws('ASC_DLBIKE_RND', 'NORMAL_ANTI')

ASC_SCTRANSIT = Beta('ASC_SCTRANSIT', 0, None, None, 0)
ASC_SCTRANSIT_S = Beta('ASC_SCTRANSIT_S', 1, None, None, 0)
ASC_SCTRANSIT_RND = ASC_SCTRANSIT + ASC_SCTRANSIT_S * bioDraws('ASC_SCTRANSIT_RND', 'NORMAL_ANTI')


In [31]:
# factor score coefficients
B_F4_DLBIKE = Beta('B_F4_DLBIKE', 0, None, None, 0)
B_F5_DLBIKE = Beta('B_F5_DLBIKE', 0, None, None, 0)
B_F6_DLBIKE = Beta('B_F6_DLBIKE', 0, None, None, 0)

B_F4_DKBIKE = Beta('B_F4_DKBIKE', 0, None, None, 0)
B_F5_DKBIKE = Beta('B_F5_DKBIKE', 0, None, None, 0)
B_F6_DKBIKE = Beta('B_F6_DKBIKE', 0, None, None, 0)

B_F4_SCTRANSIT = Beta('B_F4_SCTRANSIT', 0, None, None, 0)
B_F5_SCTRANSIT = Beta('B_F5_SCTRANSIT', 0, None, None, 0)
B_F6_SCTRANSIT = Beta('B_F6_SCTRANSIT', 0, None, None, 0)

In [32]:
# trip purpose coefficients
B_DLBIKE_NHB = Beta('B_DLBIKE_NHB', 0, None, None, 0)
B_DKBIKE_NHB = Beta('B_DKBIKE_NHB', 0, None, None, 0)
B_SCTRANSIT_NHB = Beta('B_SCTRANSIT_NHB', 0, None, None, 0)

In [33]:
V7 = (ASC_SCOOTER
      + B_SCOOTERTIME*sctime
      + B_COST_ADJ*sccost_adj
      + B_ACCESS*SCAW*(1-AVtech)
      + B_WAITAV*SCAV*AVtech
      )

V8 = (ASC_DLBIKE_RND
      + B_BIKETIME*dltime
      + B_COST_ADJ*dlcost_adj
      + B_ACCESS*DLAW
      + B_F4_DLBIKE*Factor_4
      + B_F5_DLBIKE*Factor_5
      + B_F6_DLBIKE*Factor_6
      + B_DLBIKE_NHB*trippurp_NHB
      )

V9 = (ASC_DKBIKE_RND
      + B_BIKETIME*dbtime
      + B_COST_ADJ*dbcost_adj
      + B_ACCESS*DBAW
      + B_DROP*DBDW
      + B_F4_DKBIKE*Factor_4
      + B_F5_DKBIKE*Factor_5
      + B_F6_DKBIKE*Factor_6
      + B_DKBIKE_NHB*trippurp_NHB
      )

V10 = (ASC_SCTRANSIT_RND
       + B_SCTRANSITTIME*sttotime
       + B_COST_ADJ*sttocost_adj
       + B_ACCESS*STAW*(1-AVtech)
       + B_WAITAV*STAV*AVtech
       + B_F4_SCTRANSIT*Factor_4
       + B_F5_SCTRANSIT*Factor_5
       + B_F6_SCTRANSIT*Factor_6
       + B_SCTRANSIT_NHB*trippurp_NHB
       )


In [34]:

V = {7: V7, 8: V8, 9: V9, 10: V10}
av = {7: scooter_av, 8: dlbike_av, 9: dkbike_av, 10: sctransit_av}
    

condprob = models.logit(V, av, choice)
condlikeindiv = PanelLikelihoodTrajectory(condprob)
logprob = log(MonteCarlo(condlikeindiv))

biogeme = bio.BIOGEME(database, logprob, numberOfDraws = 1000, numberOfThreads = 100)
biogeme.modelName = "MixedLogit with Factor Scores, trip purpose -Micro Only 1000 draws"

results = biogeme.estimate()
pandasResults = results.getEstimatedParameters()
print(f"Estimated betas: {len(results.data.betaValues)}")
print(f"Final log likelihood: {results.data.logLike:.3f}")
print(pandasResults.round(2))